Description
--

We need a Google Cloud Platform project with billing enabled.</br>
We first have to enable the `Cloud AI Platform Models API` and the `Compute Engine API`.</br>
We then create an AI Platform Notebooks instance, then we select the latest `PyTorch` instance type (without/with ?????To determine?????? GPUs).</br>
Once the instance has been created, we select `Open JupyterLab`:</br></br>
After setting up the environment, we run the folling commands :

In [1]:
!gsutil version -l

gsutil version: 5.6
checksum: 0988591864f8c3a8ca1ae43f519e6d15 (OK)
boto version: 2.49.0
python version: 3.8.11 (default, Dec 23 2021, 04:09:42) [Clang 12.0.1 ]
OS: Linux 4.19.0-18-cloud-amd64
multiprocessing available: True
using cloud sdk: True
pass cloud sdk credentials to gsutil: True
config path(s): /etc/boto.cfg
gsutil path: /usr/lib/google-cloud-sdk/bin/gsutil
compiled crcmod: True
installed via package manager: False
editable install: False


In [2]:
GCP_PROJECT = 'My First Project'
BUCKET_URL = 'gs://storage_bucket_speech'

In [3]:
!gsutil mb $BUCKET_URL

Creating gs://storage_bucket_speech/...
ServiceException: 409 A Cloud Storage bucket named 'storage_bucket_speech' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


The bucket already exists because it was already created

 Store the data on Google Storage
 ---

In [4]:
train_path = '../feature-extraction/train/'
test_path  = '../feature-extraction/test/'
valid_path = '../feature-extraction/valid/'

In [5]:
!gsutil -m -q cp -r $train_path $BUCKET_URL
!gsutil -m -q cp -r $test_path $BUCKET_URL
!gsutil -m -q cp -r $valid_path $BUCKET_URL

Setup for local training :
---

In [5]:
REGION = "us-west1-b"
SERVICE_ACCOUNT = "156920671469-compute@developer.gserviceaccount.com"

In [6]:
content_name = "pt-img-cls-gpu-cust"

In [7]:
! ls trainer

Dockerfile   __pycache__  dataset.py	 model.py	   task.py
__init__.py  config.py	  experiment.py  requirements.txt  utils.py


In [8]:
! cat trainer/requirements.txt

'torch==1.5.0'
'tensorboard==2.5.0'
'scikit-learn>=0.20'
'seaborn==0.11.2'

In [14]:
! pip3 install -r trainer/requirements.txt

In [15]:
%run trainer/task.py

FileNotFoundError: [Errno 2] No such file or directory: '/home/jupyter/Speech-Emotion-Recognition/gcp/output/logs/trainer.log'